In [4]:
import json
import torch

with open("./dataset/gpt2-medium_Pile_5000_25normalized/config.json") as f:
    config = json.load(f)

file_num = config["file_num"]

for i in range(file_num):
    tens = torch.load(f"./dataset/gpt2-medium_Pile_5000_25normalized/tensor_{i}.pt")
    print(tens.device)
    data = tens[:, :-2]
    norms = torch.norm(data, dim=1, keepdim=True)
    ntens = torch.div(data, norms)

    tens = torch.cat((ntens, tens[:, -2:]), dim=1)

    torch.save(tens, f"./dataset/gpt2-medium_Pile_5000_25normalized/tensor_{i}.pt")



cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0


In [5]:
from configclasses import TaskConfig


config = TaskConfig(
    subject_model_name='gpt2-medium',
    input_corpus_dataset_name='Pile',
    input_corpus_datapoint_num=5000,
    context_window=25,
)

from classifier import *
from embeddings import LoadEmbeddingDataset
from configclasses import TrainResult

clsfer = LayerClassifier()
dataset = LoadEmbeddingDataset(config.task_name() + 'normalized')

loss, acc = clsfer.train(dataset)

result = TrainResult(
    task_config=config,
    train_loss=loss,
    test_accuracy=acc,
)

Checking ./dataset/gpt2-medium_Pile_5000_25normalized...
file_num: 12, tensor_num_per_file: 260417


Checking files: 100%|██████████| 12/12 [00:00<00:00, 35951.18it/s]

./dataset/gpt2-medium_Pile_5000_25normalized is complete



Training Progress: 100%|██████████| 1000/1000 [33:07<00:00,  1.99s/it]


In [7]:
with open(f'./result/{config.task_name()}normalized.json', 'w') as f:
    json.dump({
        'task_name': config.task_name(),
        'train_loss': result.train_loss,
        'test_accuracy': result.test_accuracy
    }, f)

clsfer.interpret_model.save(f'./result/IM_{config.task_name()}normalized.model')

Model saved to ./result/IM_gpt2-medium_Pile_5000_25normalized.model
